In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

###Combining Filters
Like factors, filters can be combined. Combining filters is done using the `&` (and) and `|` (or) operators. For example, let's say we want to screen for securities that are in the top 10% of average dollar volume and have a latest close price above $20. To start, let's make a high dollar volume filter using an `AverageDollarVolume` factor and `percentile_between`:

In [2]:
dollar_volume = AverageDollarVolume(window_length=30)
high_dollar_volume = dollar_volume.percentile_between(90, 100)

Note: `percentile_between` is a `Factor` method returning a `Filter`.

Next, let's create a `latest_close` factor and define a filter for securities that closed above $20:

In [3]:
latest_close = USEquityPricing.close.latest
above_20 = latest_close > 20

Now we can combine our `high_dollar_volume` filter with our `above_20` filter using the `&` operator:

In [4]:
tradeable_filter = high_dollar_volume & above_20

This filter will evaluate to `True` for securities where both `high_dollar_volume` and `above_20` are `True`. Otherwise, it will evaluate to `False`. A similar computation can be made with the `|` (or) operator.

If we want to use this filter as a screen in our pipeline, we can simply pass `tradeable_filter` as the `screen` argument.

In [5]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = dollar_volume.percentile_between(90, 100)

    latest_close = USEquityPricing.close.latest
    above_20 = latest_close > 20

    tradeable_filter = high_dollar_volume & above_20

    return Pipeline(
        columns={
            'percent_difference': percent_difference
        },
        screen=tradeable_filter
    )

When we run this, our pipeline output now only includes ~700 securities.

In [6]:
result = run_pipeline(make_pipeline(), '2018-05-05', '2018-05-05')
print 'Number of securities that passed the filter: %d' % len(result)
result

Number of securities that passed the filter: 776


percent_difference
2018-05-07 00:00:00+00:00 Equity(24 [AAPL])              -0.008472
                          Equity(53 [ABMD])               0.035502
                          Equity(62 [ABT])               -0.004917
                          Equity(67 [ADSK])              -0.012542
                          Equity(76 [TAP])               -0.047784
                          Equity(114 [ADBE])              0.001488
                          Equity(122 [ADI])              -0.019998
                          Equity(128 [ADM])               0.013590
                          Equity(161 [AEP])               0.011888
                          Equity(168 [AET])               0.020595
                          Equity(185 [AFL])               0.016725
                          Equity(216 [HES])               0.059710
                          Equity(239 [AIG])               0.011225
                          Equity(300 [ALK])               0.033350
                          Equity(337 [AMAT])             -0.056599
                          Equity(353 [AME])              -0.037491
                          Equity(357 [TWX])              -0.007388
                          Equity(368 [AMGN])              0.003039
                          Equity(438 [AON])               0.006778
                          Equity(448 [APA])               0.024684
                          Equity(455 [APC])               0.050357
                          Equity(460 [APD])               0.007157
                          Equity(465 [APH])              -0.011440
                          Equity(630 [ADP])               0.026244
                          Equity(679 [AXP])               0.038507
                          Equity(693 [AZO])               0.003597
                          Equity(698 [BA])                0.009164
                          Equity(700 [BAC])              -0.000252
                          Equity(734 [BAX])               0.037050
                          Equity(754 [BBY])               0.040386
...                                                            ...
                          Equity(49176 [TRU])             0.084697
                          Equity(49183 [WRK])            -0.025863
                          Equity(49210 [CC])              0.008973
                          Equity(49229 [KHC])            -0.048154
                          Equity(49242 [PYPL])           -0.019813
                          Equity(49288 [LITE])           -0.090482
                          Equity(49315 [Z])              -0.038168
                          Equity(49472 [XLRE])            0.000716
                          Equity(49608 [MTCH])           -0.040528
                          Equity(49610 [SQ])             -0.015138
                          Equity(49655 [TEAM])           -0.001882
                          Equity(49751 [AVXS])            0.175539
                          Equity(50049 [FTV])            -0.032214
                          Equity(50077 [TWLO])            0.051584
                          Equity(50242 [DVMT])           -0.010284
                          Equity(50338 [NTNX])            0.007493
                          Equity(50392 [YUMC])           -0.006237
                          Equity(50428 [AA])              0.019245
                          Equity(50430 [LW])              0.044292
                          Equity(50499 [EVHC])           -0.010544
                          Equity(50524 [UWT])             0.079071
                          Equity(50716 [DXC])             0.003049
                          Equity(50758 [OKTA])            0.058217
                          Equity(50860 [VST])             0.049070
                          Equity(51012 [BKR])             0.101000
                          Equity(51157 [DD])             -0.010687
                          Equity(51231 [ROKU])           -0.000144
                          Equity(51689 [PAGS])           -0.063878
                          Eq

In the next lesson, we'll look at masking factors and filters.